<a href="https://colab.research.google.com/github/ajit-ai/DataScience/blob/main/LSTM/lstmpredition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow

In [2]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,roc_auc_score,confusion_matrix,accuracy_score,f1_score,roc_curve
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.layers import Activation, Dense, Dropout, Embedding, LSTM
import re
from IPython.display import display
import os
import string
import time
import random
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import imdb
random.seed(10)

In [3]:
num_words = 5000
( X_train , y_train ),( X_test , y_test ) = imdb.load_data(num_words = 5000)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
sequence_length = 300
batch_size = 128
X_train_seq = pad_sequences( X_train, maxlen = sequence_length)
X_test_seq = pad_sequences( X_test, maxlen = sequence_length)

In [5]:
encoder = LabelEncoder()
encoder.fit(y_train)
y_train_transformed = encoder.transform(y_train).reshape(-1,1)
y_test_transformed = encoder.transform(y_test).reshape(-1,1)

In [9]:
model = Sequential()
model.add(Embedding(num_words, 128))
model.add(LSTM( 128 , dropout = 0.25, recurrent_dropout = 0.25))
model.add(Dense(1, activation = 'sigmoid' ))
model.summary()
model.compile( optimizer = "adam" , loss = 'binary_crossentropy' , metrics = ['accuracy'] )
early_stopper = EarlyStopping( monitor = 'val_acc' , min_delta = 0.0005, patience = 3 )
reduce_lr = ReduceLROnPlateau( monitor = 'val_loss' , patience = 2 , cooldown = 0)
callbacks = [ reduce_lr , early_stopper]
train_history = model.fit( X_train_seq , y_train_transformed , batch_size = batch_size, epochs = 2,validation_split = 0.1 , verbose = 1 , callbacks = callbacks)
score = model.evaluate( X_test_seq , y_test_transformed , batch_size = batch_size)
print( "Accuracy: {:0.4}".format( score[1] ))
print( "Loss:", score[0] )

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/2
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6783 - loss: 0.5754

ValueError: EarlyStopping callback received monitor=val_acc but Keras isn't able to automatically determine whether that metric should be maximized or minimized. Pass `mode='max'` in order to do early stopping based on the highest metric value, or pass `mode='min'` in order to use the lowest value.

In [10]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error

# Generate sample time series data (replace with your dataset)
def create_sample_data(n_points=1000):
    t = np.linspace(0, 10, n_points)
    data = np.sin(t) + 0.1 * np.random.randn(n_points)  # Sine wave with noise
    return data

# Prepare data for LSTM
def prepare_data(data, look_back=60):
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_scaled = scaler.fit_transform(data.reshape(-1, 1))

    X, y = [], []
    for i in range(len(data_scaled) - look_back):
        X.append(data_scaled[i:(i + look_back), 0])
        y.append(data_scaled[i + look_back, 0])

    X = np.array(X)
    y = np.array(y)

    # Reshape X to [samples, time steps, features]
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    # Split into train and test
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    return X_train, X_test, y_train, y_test, scaler

# Build LSTM model
def build_model(look_back):
    model = Sequential()
    model.add(LSTM(50, input_shape=(look_back, 1), return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# Main execution
if __name__ == "__main__":
    # Parameters
    look_back = 60  # Number of previous time steps to use for prediction
    epochs = 50
    batch_size = 32

    # Generate or load data
    data = create_sample_data()

    # Prepare data
    X_train, X_test, y_train, y_test, scaler = prepare_data(data, look_back)

    # Build and train model
    model = build_model(look_back)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

    # Make predictions
    train_predict = model.predict(X_train)
    test_predict = model.predict(X_test)

    # Inverse transform predictions
    train_predict = scaler.inverse_transform(train_predict)
    y_train_inv = scaler.inverse_transform([y_train])
    test_predict = scaler.inverse_transform(test_predict)
    y_test_inv = scaler.inverse_transform([y_test])

    # Calculate RMSE
    train_rmse = np.sqrt(mean_squared_error(y_train_inv[0], train_predict[:,0]))
    test_rmse = np.sqrt(mean_squared_error(y_test_inv[0], test_predict[:,0]))
    print(f'Train RMSE: {train_rmse:.4f}')
    print(f'Test RMSE: {test_rmse:.4f}')

    # Predict next value
    last_sequence = X_test[-1:]  # Get the last sequence
    next_pred = model.predict(last_sequence)
    next_pred = scaler.inverse_transform(next_pred)[0, 0]
    print(f'Predicted next value: {next_pred:.4f}')

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 4s 50ms/step - loss: 0.1553
Epoch 2/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 72ms/step - loss: 0.0067
Epoch 3/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 79ms/step - loss: 0.0025
Epoch 4/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 0.0024
Epoch 5/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - loss: 0.0025
Epoch 6/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - loss: 0.0023
Epoch 7/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 0.0022
Epoch 8/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - loss: 0.0023
Epoch 9/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - loss: 0.0023
Epoch 10/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - loss: 0.0023
Epoch 11/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - loss: 0.0023
Epoch 12/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 0.0021
Epoch 13/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 0.0022
Epoch 14/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 0.0021
Epoch 15/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 0.0022
Epoch 16/50
24/

In [11]:
# prompt: LSTM top level solution for noise problem

# Predict next value
last_sequence = X_test[-1:]  # Get the last sequence
next_pred = model.predict(last_sequence)
next_pred = scaler.inverse_transform(next_pred)[0, 0]
print(f'Predicted next value: {next_pred:.4f}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Predicted next value: -0.4909
